In [2]:
!pip install catboost
!pip install feature-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.6/378.6 kB 4.7 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, FunctionTransformer
from sklearn.metrics import classification_report, confusion_matrix

from feature_engine.selection import DropConstantFeatures, SmartCorrelatedSelection

In [4]:
import xgboost
xgboost.__version__

'2.1.4'

In [5]:
path = "/content/drive/MyDrive/ml-project/data"

In [6]:
FILE = "smart-contract-dataset.csv"

In [7]:
data = pd.read_csv(os.path.join(path, FILE))

In [8]:
data

,Unnamed: 0,hash_id,label,bytecode_len,Weight bytecode_character_6,Weight bytecode_character_0,Weight bytecode_character_8,Weight bytecode_character_4,Weight bytecode_character_5,Weight bytecode_character_2,Weight bytecode_character_1,Weight bytecode_character_a,Weight bytecode_character_7,Weight bytecode_character_3,Weight bytecode_character_f,Weight bytecode_character_9,Weight bytecode_character_b,Weight bytecode_character_c,Weight bytecode_character_d,Weight bytecode_character_e,bytecode_character_6,bytecode_character_0,bytecode_character_8,bytecode_character_4,bytecode_character_5,bytecode_character_2,bytecode_character_1,bytecode_character_a,bytecode_character_7,bytecode_character_3,bytecode_character_f,bytecode_character_9,bytecode_character_b,bytecode_character_c,bytecode_character_d,bytecode_character_e,bytecode_entropy,ast_len_exportedSymbols,ast_id,ast_nodetype,ast_src,ast_len_nodes,Opcode weight STOP,Opcode weight ADD,Opcode weight MUL,Opcode weight SUB,Opcode weight DIV,Opcode weight SDIV,Opcode weight MOD,Opcode weight SMOD,Opcode weight ADDMOD,Opcode weight MULMOD,Opcode weight EXP,Opcode weight SIGNEXTEND,Opcode weight LT,Opcode weight GT,Opcode weight SLT,Opcode weight SGT,Opcode weight EQ,Opcode weight ISZERO,Opcode weight AND,Opcode weight OR,Opcode weight XOR,Opcode weight NOT,Opcode weight BYTE,Opcode weight SHL,Opcode weight SHR,Opcode weight SAR,Opcode weight SHA3,Opcode weight ADDRESS,Opcode weight BALANCE,Opcode weight ORIGIN,Opcode weight CALLER,Opcode weight CALLVALUE,Opcode weight CALLDATALOAD,Opcode weight CALLDATASIZE,Opcode weight CALLDATACOPY,Opcode weight CODESIZE,Opcode weight CODECOPY,Opcode weight GASPRICE,Opcode weight EXTCODESIZE,Opcode weight EXTCODECOPY,Opcode weight RETURNDATASIZE,Opcode weight RETURNDATACOPY,Opcode weight BLOCKHASH,Opcode weight COINBASE,Opcode weight TIMESTAMP,Opcode weight NUMBER,Opcode weight DIFFICULTY,Opcode weight GASLIMIT,Opcode weight POP,Opcode weight MLOAD,Opcode weight MSTORE,Opcode weight MSTORE8,Opcode weight SLOAD,Opcode weight SSTORE,Opcode weight JUMP,Opcode weight JUMPI,Opcode weight PC,Opcode weight MSIZE,Opcode weight GAS,Opcode weight JUMPDEST,Opcode weight PUSH1,Opcode weight PUSH2,Opcode weight PUSH3,Opcode weight PUSH4,Opcode weight PUSH5,Opcode weight PUSH6,Opcode weight PUSH7,Opcode weight PUSH8,Opcode weight PUSH9,Opcode weight PUSH10,Opcode weight PUSH11,Opcode weight PUSH12,Opcode weight PUSH13,Opcode weight PUSH14,Opcode weight PUSH15,Opcode weight PUSH16,Opcode weight PUSH17,Opcode weight PUSH18,Opcode weight PUSH19,Opcode weight PUSH20,Opcode weight PUSH21,Opcode weight PUSH22,Opcode weight PUSH23,Opcode weight PUSH24,Opcode weight PUSH25,Opcode weight PUSH26,Opcode weight PUSH27,Opcode weight PUSH28,Opcode weight PUSH29,Opcode weight PUSH30,Opcode weight PUSH31,Opcode weight PUSH32,Opcode weight DUP1,Opcode weight DUP2,Opcode weight DUP3,Opcode weight DUP4,Opcode weight DUP5,Opcode weight DUP6,Opcode weight DUP7,Opcode weight DUP8,Opcode weight DUP9,Opcode weight DUP10,Opcode weight DUP11,Opcode weight DUP12,Opcode weight DUP13,Opcode weight DUP14,Opcode weight DUP15,Opcode weight DUP16,Opcode weight SWAP1,Opcode weight SWAP2,Opcode weight SWAP3,Opcode weight SWAP4,Opcode weight SWAP5,Opcode weight SWAP6,Opcode weight SWAP7,Opcode weight SWAP8,Opcode weight SWAP9,Opcode weight SWAP10,Opcode weight SWAP11,Opcode weight SWAP12,Opcode weight SWAP13,Opcode weight SWAP14,Opcode weight SWAP15,Opcode weight SWAP16,Opcode weight LOG0,Opcode weight LOG1,Opcode weight LOG2,Opcode weight LOG3,Opcode weight LOG4,Opcode weight CREATE,Opcode weight CALL,Opcode weight CALLCODE,Opcode weight RETURN,Opcode weight DELEGATECALL,Opcode weight STATICCALL,Opcode weight REVERT,Opcode weight INVALID,Opcode weight SELFDESTRUCT,from,require,dev,internal,string,view,mapping,sub,emit,length,pure,will,not,approve,external,memory,eth,else,can,calls,data,q,Weight bytecode_character__,Weight bytecode_character_<,Weight bytecode_character_s,Weight bytecode_character_t,We

In [9]:
data = data.rename(
    columns=lambda col: (
        col
        .replace('<', 'lt')
        .replace('>', 'gt')
        .replace('__', '_double_underscore')
        .replace(':', '_colon')
    )
)

In [9]:
data.label.value_counts()

,count
label,
0,26915
1,9756


In [10]:
data.describe()

,Unnamed_colon 0,label,bytecode_len,Weight bytecode_character_6,Weight bytecode_character_0,Weight bytecode_character_8,Weight bytecode_character_4,Weight bytecode_character_5,Weight bytecode_character_2,Weight bytecode_character_1,Weight bytecode_character_a,Weight bytecode_character_7,Weight bytecode_character_3,Weight bytecode_character_f,Weight bytecode_character_9,Weight bytecode_character_b,Weight bytecode_character_c,Weight bytecode_character_d,Weight bytecode_character_e,bytecode_character_6,bytecode_character_0,bytecode_character_8,bytecode_character_4,bytecode_character_5,bytecode_character_2,bytecode_character_1,bytecode_character_a,bytecode_character_7,bytecode_character_3,bytecode_character_f,bytecode_character_9,bytecode_character_b,bytecode_character_c,bytecode_character_d,bytecode_character_e,bytecode_entropy,ast_len_exportedSymbols,ast_id,ast_len_nodes,Opcode weight STOP,Opcode weight ADD,Opcode weight MUL,Opcode weight SUB,Opcode weight DIV,Opcode weight SDIV,Opcode weight MOD,Opcode weight SMOD,Opcode weight ADDMOD,Opcode weight MULMOD,Opcode weight EXP,Opcode weight SIGNEXTEND,Opcode weight LT,Opcode weight GT,Opcode weight SLT,Opcode weight SGT,Opcode weight EQ,Opcode weight ISZERO,Opcode weight AND,Opcode weight OR,Opcode weight XOR,Opcode weight NOT,Opcode weight BYTE,Opcode weight SHL,Opcode weight SHR,Opcode weight SAR,Opcode weight SHA3,Opcode weight ADDRESS,Opcode weight BALANCE,Opcode weight ORIGIN,Opcode weight CALLER,Opcode weight CALLVALUE,Opcode weight CALLDATALOAD,Opcode weight CALLDATASIZE,Opcode weight CALLDATACOPY,Opcode weight CODESIZE,Opcode weight CODECOPY,Opcode weight GASPRICE,Opcode weight EXTCODESIZE,Opcode weight EXTCODECOPY,Opcode weight RETURNDATASIZE,Opcode weight RETURNDATACOPY,Opcode weight BLOCKHASH,Opcode weight COINBASE,Opcode weight TIMESTAMP,Opcode weight NUMBER,Opcode weight DIFFICULTY,Opcode weight GASLIMIT,Opcode weight POP,Opcode weight MLOAD,Opcode weight MSTORE,Opcode weight MSTORE8,Opcode weight SLOAD,Opcode weight SSTORE,Opcode weight JUMP,Opcode weight JUMPI,Opcode weight PC,Opcode weight MSIZE,Opcode weight GAS,Opcode weight JUMPDEST,Opcode weight PUSH1,Opcode weight PUSH2,Opcode weight PUSH3,Opcode weight PUSH4,Opcode weight PUSH5,Opcode weight PUSH6,Opcode weight PUSH7,Opcode weight PUSH8,Opcode weight PUSH9,Opcode weight PUSH10,Opcode weight PUSH11,Opcode weight PUSH12,Opcode weight PUSH13,Opcode weight PUSH14,Opcode weight PUSH15,Opcode weight PUSH16,Opcode weight PUSH17,Opcode weight PUSH18,Opcode weight PUSH19,Opcode weight PUSH20,Opcode weight PUSH21,Opcode weight PUSH22,Opcode weight PUSH23,Opcode weight PUSH24,Opcode weight PUSH25,Opcode weight PUSH26,Opcode weight PUSH27,Opcode weight PUSH28,Opcode weight PUSH29,Opcode weight PUSH30,Opcode weight PUSH31,Opcode weight PUSH32,Opcode weight DUP1,Opcode weight DUP2,Opcode weight DUP3,Opcode weight DUP4,Opcode weight DUP5,Opcode weight DUP6,Opcode weight DUP7,Opcode weight DUP8,Opcode weight DUP9,Opcode weight DUP10,Opcode weight DUP11,Opcode weight DUP12,Opcode weight DUP13,Opcode weight DUP14,Opcode weight DUP15,Opcode weight DUP16,Opcode weight SWAP1,Opcode weight SWAP2,Opcode weight SWAP3,Opcode weight SWAP4,Opcode weight SWAP5,Opcode weight SWAP6,Opcode weight SWAP7,Opcode weight SWAP8,Opcode weight SWAP9,Opcode weight SWAP10,Opcode weight SWAP11,Opcode weight SWAP12,Opcode weight SWAP13,Opcode weight SWAP14,Opcode weight SWAP15,Opcode weight SWAP16,Opcode weight LOG0,Opcode weight LOG1,Opcode weight LOG2,Opcode weight LOG3,Opcode weight LOG4,Opcode weight CREATE,Opcode weight CALL,Opcode weight CALLCODE,Opcode weight RETURN,Opcode weight DELEGATECALL,Opcode weight STATICCALL,Opcode weight REVERT,Opcode weight INVALID,Opcode weight SELFDESTRUCT,from,require,dev,internal,string,view,mapping,sub,emit,length,pure,will,not,approve,external,memory,eth,else,can,calls,data,q,Weight bytecode_character_double_underscore,Weight bytecode_character_lt,Weight bytecode_character_s,Weight bytecode_character_t,Weight by

In [10]:
X = data.iloc[:, 3:].select_dtypes(include="number")
y = data.label

In [11]:
SEED = 78

In [12]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=SEED
)

print(X_train.shape, X_val.shape, X_test.shape)

(23468, 263) (5868, 263) (7335, 263)


### Naive Bayes

In [14]:
pipeline_naive_bayes = Pipeline(
    [
        ('gnb', GaussianNB())
    ]
)

pipeline_naive_bayes.fit(X_train, y_train)
y_pred = pipeline_naive_bayes.predict(X_val)

In [15]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.81      0.79      4238
           1       0.41      0.34      0.38      1630

    accuracy                           0.68      5868
   macro avg       0.59      0.58      0.58      5868
weighted avg       0.67      0.68      0.67      5868



In [16]:
confusion_matrix(y_val, y_pred)

array([[3444,  794],
       [1070,  560]])

### Decision Trees

In [17]:
decicion_tree_hyperparameters = {
    "max_depth": 15,
    "class_weight": "balanced",
    "random_state": SEED
}

decision_tree = DecisionTreeClassifier(
    **decicion_tree_hyperparameters
)


decision_tree_pipeline = Pipeline(
    [
        ("model", decision_tree)
    ]
)

In [18]:
decision_tree_pipeline.fit(X_train, y_train)
y_pred = decision_tree_pipeline.predict(X_val)

In [19]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.76      0.80      4238
           1       0.50      0.61      0.55      1630

    accuracy                           0.72      5868
   macro avg       0.67      0.69      0.67      5868
weighted avg       0.74      0.72      0.73      5868



In [20]:
confusion_matrix(y_val, y_pred)

array([[3230, 1008],
       [ 637,  993]])

### Logistic Regression

In [21]:
logistic_regression_hyperparameters = {
    'penalty': 'l2',
    'C': 1.0,
    'solver': 'lbfgs',
    'max_iter': 1000,
    'class_weight': 'balanced',
    'random_state': SEED
}

logistic_regression = LogisticRegression(
    **logistic_regression_hyperparameters
)

logistic_regression_pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("model", logistic_regression)
    ]
)

In [22]:
logistic_regression_pipeline.fit(X_train, y_train)
y_pred = logistic_regression_pipeline.predict(X_val)

In [23]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.75      0.78      4238
           1       0.45      0.53      0.49      1630

    accuracy                           0.69      5868
   macro avg       0.63      0.64      0.63      5868
weighted avg       0.71      0.69      0.70      5868



In [24]:
confusion_matrix(y_val, y_pred)

array([[3195, 1043],
       [ 771,  859]])

### Linear SVC

In [25]:
linear_svc_hyperparameters = {
    'C': 1.0,
    'tol': 1e-4,
    'max_iter': 1000,
    'class_weight': 'balanced',
    'random_state': SEED
}

linear_svc = LinearSVC(
    **linear_svc_hyperparameters
)

linear_svc_pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("model", linear_svc)
    ]
)

In [26]:
linear_svc_pipeline.fit(X_train, y_train)
y_pred = linear_svc_pipeline.predict(X_val)

In [27]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.75      0.78      4238
           1       0.45      0.53      0.49      1630

    accuracy                           0.69      5868
   macro avg       0.63      0.64      0.63      5868
weighted avg       0.71      0.69      0.70      5868



In [28]:
confusion_matrix(y_val, y_pred)

array([[3189, 1049],
       [ 772,  858]])

### Support Vector Machine

In [29]:
svm_hyperparameters = {
    'C': 1.0,
    'kernel': 'rbf',
    'gamma': 'scale',
    'tol': 1e-3,
    'max_iter': 10000,
    'class_weight': 'balanced',
    'random_state': SEED
}

svm = SVC(
    **svm_hyperparameters
)

svm_pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("model", svm)
    ]
)

In [30]:
svm_pipeline.fit(X_train, y_train)
y_pred = svm_pipeline.predict(X_val)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [31]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.18      0.29      4238
           1       0.26      0.73      0.38      1630

    accuracy                           0.34      5868
   macro avg       0.45      0.46      0.33      5868
weighted avg       0.54      0.34      0.31      5868



In [32]:
confusion_matrix(y_val, y_pred)

array([[ 783, 3455],
       [ 434, 1196]])

### Random Forest

In [33]:
random_forest_hyperparameters = {
    'n_estimators': 100,
    'criterion': 'gini',
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'bootstrap': True,
    'class_weight': 'balanced',
    'random_state': SEED
}

random_forest = RandomForestClassifier(
    **random_forest_hyperparameters
)

random_forest_pipeline = Pipeline(
    [
        ("model", random_forest)
    ]
)

In [34]:
random_forest_pipeline.fit(X_train, y_train)
y_pred = random_forest_pipeline.predict(X_val)

In [35]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82      4238
           1       0.53      0.61      0.57      1630

    accuracy                           0.74      5868
   macro avg       0.69      0.70      0.69      5868
weighted avg       0.76      0.74      0.75      5868



In [36]:
confusion_matrix(y_val, y_pred)

array([[3374,  864],
       [ 639,  991]])

### Adaboost

In [37]:
adaboost_hyperparameters = {
    'estimator': DecisionTreeClassifier(max_depth=3, class_weight='balanced'),
    'n_estimators': 50,
    'learning_rate': 1.0,
    'random_state': SEED
}

adaboost = AdaBoostClassifier(
    **adaboost_hyperparameters
)

adaboost_pipeline = Pipeline(
    [
        ("model", adaboost)
    ]
)

In [38]:
adaboost_pipeline.fit(X_train, y_train)
y_pred = adaboost_pipeline.predict(X_val)

In [39]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.81      0.81      4238
           1       0.51      0.51      0.51      1630

    accuracy                           0.73      5868
   macro avg       0.66      0.66      0.66      5868
weighted avg       0.73      0.73      0.73      5868



In [40]:
confusion_matrix(y_val, y_pred)

array([[3428,  810],
       [ 792,  838]])

### XGBoost

In [41]:
xgboost_hyperparameters = {
    "objective": "binary:logistic",
    "eval_metric": "aucpr",
    "scale_pos_weight": 3,
    "learning_rate": 0.05,
    "max_depth": 6,
    "min_child_weight": 1,
    "gamma": 0.2,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "lambda": 1.0,
    "alpha": 0.0,
    "n_estimators": 500,
    # "early_stopping_rounds": 50,
    # "use_label_encoder": False,
    "verbosity": 1,
    "random_state": SEED
}

xgboost = XGBClassifier(**xgboost_hyperparameters)

pipeline_xgboost = Pipeline(
    [
        ("model", xgboost)
    ]
)

In [42]:
pipeline_xgboost.fit(X_train, y_train)
y_pred = pipeline_xgboost.predict(X_val)

In [43]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.80      0.82      4238
           1       0.55      0.64      0.59      1630

    accuracy                           0.75      5868
   macro avg       0.70      0.72      0.71      5868
weighted avg       0.77      0.75      0.76      5868



In [44]:
print(confusion_matrix(y_val, y_pred))

[[3380  858]
 [ 586 1044]]


### LightGBM

In [45]:
lightgbm_hyperparameters = {
    "objective": "binary",
    "metric": "aucpr",
    "is_unbalance": True,
    "learning_rate": 0.05,
    "num_leaves": 31,
    "max_depth": -1,
    "min_child_samples": 20,
    "min_child_weight": 1e-3,
    "subsample": 0.8,
    "subsample_freq": 1,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.0,
    "reg_lambda": 1.0,
    "n_estimators": 500,
    # "early_stopping_round": 50,
    "verbosity": -1,
    "random_state": SEED
}

lightgbm = LGBMClassifier(**lightgbm_hyperparameters)

pipeline_lightgbm = Pipeline(
    [
        ("model", lightgbm)
    ]
)

In [46]:
pipeline_lightgbm.fit(X_train, y_train)
y_pred = pipeline_lightgbm.predict(X_val)

In [47]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.81      0.83      4238
           1       0.55      0.63      0.59      1630

    accuracy                           0.76      5868
   macro avg       0.70      0.72      0.71      5868
weighted avg       0.77      0.76      0.76      5868



In [48]:
print(confusion_matrix(y_val, y_pred))

[[3412  826]
 [ 610 1020]]


### CatBoost

In [49]:
catboost_hyperparaters = {
    "iterations": 1000,
    "learning_rate": 0.05,
    "depth": 6,
    "l2_leaf_reg": 3,
    "random_strength": 1.5,
    "bagging_temperature": 1.0,
    "border_count": 128,
    "loss_function": "Logloss",
    "eval_metric": "F1",
    "scale_pos_weight": 3,
    # "early_stopping_rounds": 50,
    "verbose": 1,
    "task_type": "CPU",
    "random_seed": SEED
}

pipeline_catboost = Pipeline(
    [
        ("model", CatBoostClassifier(**catboost_hyperparaters))
    ]
)

In [50]:
pipeline_catboost.fit(X_train, y_train)
y_pred = pipeline_catboost.predict(X_val)

0:	learn: 0.6156408	total: 170ms	remaining: 2m 49s
1:	learn: 0.6230520	total: 274ms	remaining: 2m 16s
2:	learn: 0.6263433	total: 359ms	remaining: 1m 59s
3:	learn: 0.6095758	total: 456ms	remaining: 1m 53s
4:	learn: 0.6139931	total: 554ms	remaining: 1m 50s
5:	learn: 0.6144499	total: 646ms	remaining: 1m 46s
6:	learn: 0.6128920	total: 737ms	remaining: 1m 44s
7:	learn: 0.6180719	total: 842ms	remaining: 1m 44s
8:	learn: 0.6244319	total: 948ms	remaining: 1m 44s
9:	learn: 0.6262412	total: 1.02s	remaining: 1m 40s
10:	learn: 0.6253463	total: 1.06s	remaining: 1m 35s
11:	learn: 0.6284563	total: 1.11s	remaining: 1m 31s
12:	learn: 0.6394462	total: 1.16s	remaining: 1m 28s
13:	learn: 0.6426879	total: 1.21s	remaining: 1m 25s
14:	learn: 0.6407998	total: 1.26s	remaining: 1m 22s
15:	learn: 0.6388309	total: 1.3s	remaining: 1m 20s
16:	learn: 0.6444053	total: 1.36s	remaining: 1m 18s
17:	learn: 0.6527054	total: 1.41s	remaining: 1m 17s
18:	learn: 0.6514451	total: 1.47s	remaining: 1m 15s
19:	learn: 0.6519354	to

In [51]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.77      0.81      4238
           1       0.53      0.65      0.58      1630

    accuracy                           0.74      5868
   macro avg       0.69      0.71      0.70      5868
weighted avg       0.76      0.74      0.75      5868



In [52]:
print(confusion_matrix(y_val, y_pred))

[[3274  964]
 [ 564 1066]]


### Análise Exploratória

In [53]:
def get_outlier_summary(df, iqr_multiplier=1.5, round_digits=2):
    numeric_df = df.select_dtypes(include='number')
    outlier_info = {}

    for col in numeric_df.columns:
        Q1 = numeric_df[col].quantile(0.25)
        Q3 = numeric_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - iqr_multiplier * IQR
        upper = Q3 + iqr_multiplier * IQR
        outliers = numeric_df[(numeric_df[col] < lower) | (numeric_df[col] > upper)][col]

        if not outliers.empty:
            total = len(numeric_df)
            outlier_info[col] = {
                'count': len(outliers),
                'proportion': round(len(outliers) / total, round_digits),
                'median': round(numeric_df[col].median(), round_digits),
                'outlier_min': round(outliers.min(), round_digits),
                'outlier_max': round(outliers.max(), round_digits)
            }

    return outlier_info


In [13]:
numerical_features = data.iloc[:, 3:].select_dtypes(include="number")

In [55]:
outlier_summary = get_outlier_summary(numerical_features)

In [56]:
for col, stats in outlier_summary.items():
    print(f"\n{col}")
    for k, v in stats.items():
        print(f"  {k}: {v}")



bytecode_len
  count: 1027
  proportion: 0.03
  median: 1660.0
  outlier_min: 20262
  outlier_max: 90356

Weight bytecode_character_0
  count: 1
  proportion: 0.0
  median: 0.19
  outlier_min: 0.69
  outlier_max: 0.69

Weight bytecode_character_4
  count: 59
  proportion: 0.0
  median: 0.02
  outlier_min: 0.07
  outlier_max: 0.14

Weight bytecode_character_2
  count: 1
  proportion: 0.0
  median: 0.04
  outlier_min: 0.19
  outlier_max: 0.19

Weight bytecode_character_a
  count: 4642
  proportion: 0.13
  median: 0.01
  outlier_min: 0.02
  outlier_max: 0.09

Weight bytecode_character_7
  count: 348
  proportion: 0.01
  median: 0.02
  outlier_min: 0.06
  outlier_max: 0.1

Weight bytecode_character_3
  count: 5
  proportion: 0.0
  median: 0.03
  outlier_min: 0.15
  outlier_max: 0.24

Weight bytecode_character_9
  count: 2
  proportion: 0.0
  median: 0.03
  outlier_min: 0.11
  outlier_max: 0.11

Weight bytecode_character_b
  count: 64
  proportion: 0.0
  median: 0.01
  outlier_min: 0.05
  

In [57]:
len(outlier_summary.keys())

249

In [59]:
len(numerical_features.columns)

263

In [60]:
def get_low_variance_numerical(df, threshold=1e-4):
    numeric_df = df.select_dtypes(include='number')
    low_var_cols = [col for col in numeric_df.columns if numeric_df[col].var() < threshold]
    return low_var_cols

In [61]:
low_variance_features = get_low_variance_numerical(numerical_features, threshold=1e-4)
low_variance_features

['Weight bytecode_character_c',
 'Weight bytecode_character_d',
 'Weight bytecode_character_e',
 'ast_len_exportedSymbols',
 'Opcode weight MUL',
 'Opcode weight SUB',
 'Opcode weight DIV',
 'Opcode weight SDIV',
 'Opcode weight MOD',
 'Opcode weight SMOD',
 'Opcode weight ADDMOD',
 'Opcode weight MULMOD',
 'Opcode weight EXP',
 'Opcode weight SIGNEXTEND',
 'Opcode weight LT',
 'Opcode weight GT',
 'Opcode weight SLT',
 'Opcode weight SGT',
 'Opcode weight OR',
 'Opcode weight XOR',
 'Opcode weight NOT',
 'Opcode weight BYTE',
 'Opcode weight SHL',
 'Opcode weight SHR',
 'Opcode weight SAR',
 'Opcode weight SHA3',
 'Opcode weight BALANCE',
 'Opcode weight ORIGIN',
 'Opcode weight CALLER',
 'Opcode weight CALLVALUE',
 'Opcode weight CALLDATALOAD',
 'Opcode weight CALLDATASIZE',
 'Opcode weight CALLDATACOPY',
 'Opcode weight CODESIZE',
 'Opcode weight CODECOPY',
 'Opcode weight GASPRICE',
 'Opcode weight EXTCODESIZE',
 'Opcode weight EXTCODECOPY',
 'Opcode weight RETURNDATASIZE',
 'Opcod

In [62]:
len(low_variance_features)

158

In [63]:
def get_coefficient_variation(df, exclude_cols=[], round_digits=4):
    numeric_df = df.select_dtypes(include='number')
    remaining = [col for col in numeric_df.columns if col not in exclude_cols]
    result = {}

    for col in remaining:
        mean = numeric_df[col].mean()
        std = numeric_df[col].std()
        if mean != 0:
            cv = std / abs(mean)
            result[col] = round(cv, round_digits)
        else:
            result[col] = None

    return result

def get_quartile_coefficient_dispersion(df, exclude_cols=[], round_digits=4):
    numeric_df = df.select_dtypes(include='number')
    remaining = [col for col in numeric_df.columns if col not in exclude_cols]
    result = {}

    for col in remaining:
        q1 = numeric_df[col].quantile(0.25)
        q3 = numeric_df[col].quantile(0.75)
        denominator = q3 + q1
        if denominator != 0:
            qcd = (q3 - q1) / denominator
            result[col] = round(qcd, round_digits)
        else:
            result[col] = None

    return result

def get_dispersion_metrics(df, exclude_cols=[], round_digits=4):
    cv = get_coefficient_variation(df, exclude_cols, round_digits)
    qcd = get_quartile_coefficient_dispersion(df, exclude_cols, round_digits)

    return {
        'cv': cv,
        'qcd': qcd
    }

def print_dispersion_metrics(metrics_dict):
    print("Dispersion Metrics:\n")
    print(f"{'Feature':<30} {'CV':>10} {'QCD':>10}")
    print("-" * 52)

    features = sorted(metrics_dict['cv'].keys())
    for feature in features:
        cv = metrics_dict['cv'].get(feature)
        qcd = metrics_dict['qcd'].get(feature)
        cv_str = f"{cv:.4f}" if cv is not None else "None"
        qcd_str = f"{qcd:.4f}" if qcd is not None else "None"
        print(f"{feature:<30} {cv_str:>10} {qcd_str:>10}")


In [64]:
metrics = get_dispersion_metrics(numerical_features, exclude_cols=low_variance_features)
print_dispersion_metrics(metrics)


Dispersion Metrics:

Feature                                CV        QCD
----------------------------------------------------
Opcode weight ADD                  0.9043     1.0000
Opcode weight ADDRESS              2.7409     1.0000
Opcode weight AND                  0.9905     1.0000
Opcode weight DUP1                 0.6972     1.0000
Opcode weight DUP2                 0.7255     1.0000
Opcode weight DUP3                 1.1704     1.0000
Opcode weight EQ                   1.4253     1.0000
Opcode weight ISZERO               0.8275     1.0000
Opcode weight JUMPDEST             0.6475     1.0000
Opcode weight JUMPI                0.6755     1.0000
Opcode weight MSTORE               0.7606     1.0000
Opcode weight POP                  0.7313     1.0000
Opcode weight PUSH1                0.7009     1.0000
Opcode weight PUSH2                0.8687     1.0000
Opcode weight PUSH20               0.9474     1.0000
Opcode weight REVERT               0.9379     1.0000
Opcode weight STOP       

**Duas medidas de dispersão** para cada feature numérica:
* **CV (Coeficiente de Variação)**: Desvio padrão dividido pela média. CV alto significa que os valores variam muito em relação à média.
* **QCD (Coeficiente de Dispersão Quartil)**: (Q3−Q1)/(Q3+Q1), que é robusto a outliers. Um QCD de 1 sugere dispersão muito alta, enquanto próximo de 0 indica agrupamento compacto.

1. **CV < 1** — por exemplo, `Opcode weight DUP1` (0,6972), `Weight bytecode_character_6` (0,6223)
   * Essas características são **relativamente estáveis** em torno da média.
   * Ainda podem ser úteis se os valores forem significativos para a predição, mas não variam drasticamente.
2. **CV ~ 1–2** — por exemplo, `Opcode weight EQ` (1,4253), `bytecode_character_d` (1,3922)
   * Variabilidade moderada a alta.
   * Essas características têm uma dispersão razoável e podem contribuir com poder discriminativo.
3. **CV >> 2** — por exemplo, `bytecode_character_O` (191,49), `bytecode_character_q` (44,48)
   * **Variabilidade extremamente alta**, frequentemente devido a dados **enviesados ou esparsos** (muitos zeros ou valores grandes raros).
   * Geralmente são **candidatos para transformação (por exemplo, escala logarítmica)** ou até mesmo **remoção**, especialmente se não correlacionados com nada mais.
4. **QCD = 1** para a maioria das características
   * Isso sugere que **Q1 é 0** e **Q3 é diferente de zero**, o que frequentemente acontece com **características esparsas ou infladas por zeros** (por exemplo, opcodes codificados one-hot).
   * Significa que pelo menos 75% dos valores são zero — alta esparsidade.
5. **QCD < 1** — por exemplo, `ast_id` (0,4617), `mapping` (0,1429), `string` (0,3333), `require` (0,8462)
   * Esses têm **distribuições mais equilibradas**, menos esparsas que a maioria.
   * Se o QCD estiver **próximo de 0**, então os valores estão densamente agrupados.
6. **QCD = None** — por exemplo, `bytecode_character_<`, `eth`, `memory`
   * Provavelmente devido a **quantis zero ou indefinidos** (por exemplo, valores constantes ou quase constantes, ou Q1 + Q3 = 0).
   * Você deve verificar manualmente esses casos, pois podem ser características constantes ou problemáticas.

In [65]:
def high_correlation_pairs(df, low_variance_features=None, threshold=0.9):
    if low_variance_features is None:
        num_df = df.select_dtypes(include=np.number)
    else:
        num_df = df.select_dtypes(include=np.number).drop(columns=low_variance_features, errors='ignore')

    corr_matrix = num_df.corr()
    mask = (corr_matrix.abs() >= threshold) & (corr_matrix.abs() < 1)
    pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if mask.iloc[i, j]:
                pairs.append({
                    'Feature 1': corr_matrix.columns[i],
                    'Feature 2': corr_matrix.columns[j],
                    'Correlation': corr_matrix.iloc[i, j]
                })
    return pd.DataFrame(pairs).sort_values(by='Correlation', key=abs, ascending=False).reset_index(drop=True)


In [66]:
high_correlated_features = high_correlation_pairs(numerical_features, low_variance_features=low_variance_features, threshold=0.9)
high_correlated_features

,Feature 1,Feature 2,Correlation
0,bytecode_character_M,bytecode_character_h,0.999570
1,bytecode_character_lt,bytecode_character_n,0.998887
2,bytecode_character_n,bytecode_character_gt,0.998887
3,bytecode_character_n,bytecode_character__colon,0.998887
4,bytecode_character_double_underscore,bytecode_character_lt,0.994741
5,bytecode_character_double_underscore,bytecode_character__colon,0.994741
6,bytecode_character_double_underscore,bytecode_character_gt,0.994741
7,bytecode_character_5,bytecode_character_1,0.994627
8,bytecode_character_double_underscore,bytecode_character_n,0.993805
9,bytecode_character_6,bytecode_character_1,0.992364


In [67]:
def show_feature_sparsity_excluding_low_variance(df, low_variance_features):
    filtered_df = df.drop(columns=low_variance_features, errors='ignore')
    sparsity = (filtered_df == 0).sum() / len(filtered_df)

    sparsity_df = (
        pd.DataFrame({'sparsity': sparsity})
        .sort_values(by='sparsity', ascending=False)
    )

    return sparsity_df


In [68]:
sparsity_info = show_feature_sparsity_excluding_low_variance(numerical_features, low_variance_features)
sparsity_info

,sparsity
bytecode_character_g,0.999973
bytecode_character_z,0.999973
bytecode_character_O,0.999973
bytecode_character_T,0.999973
bytecode_character_u,0.999973
bytecode_character_x,0.999945
bytecode_character_D,0.999918
bytecode_character_l,0.999918
bytecode_character_A,0.999891
bytecode_character_k,0.999891


### Pipelines

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin

class LowVarianceFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=1e-4):
        self.threshold = threshold
        self.selected_features_ = None

    def fit(self, X, y=None):
        numeric_df = X.select_dtypes(include='number')
        self.selected_features_ = [
            col for col in numeric_df.columns if numeric_df[col].var() > self.threshold
        ]
        return self

    def transform(self, X):
        if self.selected_features_ is None:
            raise RuntimeError("You must fit the transformer before transforming data.")
        return X[self.selected_features_]


In [14]:
preprocessing_pipeline = Pipeline(
    [
        (
            "LowVarianceFeatureSelector", LowVarianceFeatureSelector(
                threshold=1e-4
            )
        ),
        (
            "SmartCorrelatedSelection", SmartCorrelatedSelection(
                threshold=.9,
                missing_values='ignore',
                selection_method='variance'
            )
        ),
    ]
)

X_train_ = preprocessing_pipeline.fit_transform(X_train)

In [15]:
X_train_.shape

(23468, 56)

In [16]:
X_train.shape

(23468, 263)

In [17]:
list(X_train_.columns)

['bytecode_len',
 'Weight bytecode_character_4',
 'Weight bytecode_character_a',
 'Weight bytecode_character_f',
 'Weight bytecode_character_b',
 'bytecode_character_e',
 'bytecode_entropy',
 'ast_id',
 'ast_len_nodes',
 'Opcode weight STOP',
 'Opcode weight ADD',
 'Opcode weight ISZERO',
 'Opcode weight ADDRESS',
 'Opcode weight MSTORE',
 'Opcode weight JUMPI',
 'Opcode weight PUSH1',
 'Opcode weight DUP1',
 'Opcode weight DUP2',
 'Opcode weight DUP3',
 'Opcode weight SWAP1',
 'Opcode weight REVERT',
 'from',
 'require',
 'dev',
 'internal',
 'string',
 'view',
 'mapping',
 'sub',
 'emit',
 'length',
 'pure',
 'will',
 'not',
 'approve',
 'external',
 'memory',
 'eth',
 'else',
 'can',
 'calls',
 'data',
 'q',
 'bytecode_character_double_underscore',
 'bytecode_character_S',
 'bytecode_character_h',
 'bytecode_character_E',
 'bytecode_character_R',
 'bytecode_character_o',
 'bytecode_character_r',
 'bytecode_character_y',
 'bytecode_character_D',
 'bytecode_character_A',
 'bytecode_ch